**This notebook is for:**
1. Preprocessing variables.

In [1]:
import sys, os 
prj_path = os.path.dirname(os.getcwd())
utils_path = prj_path + '/utils'
if  not utils_path in sys.path:
    print('adding utils to path ')
    sys.path.insert(1, utils_path)

adding utils to path 


In [2]:
from utilities import *

from google_cloud import BigQuery, Storage
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime

import os
import sys

import boto3
import base64

from replib.descriptors.metaprod2vec import MetaProd2Vec

#Cliente BigQuery
AUTH_BIGQUERY = base64.b64decode(os.environ['SECRET_AUTH_BIGQUERY_MODEL'])
bq = BigQuery(AUTH_BIGQUERY)
storage =Storage(AUTH_BIGQUERY)

#Load config.
files = glob.glob(f"{utils_path}/*.yml") 
conf = load_conf(files)

In [ ]:
# !pip install --index-url https://pypi.artifacts.furycloud.io replib.descriptors.metaprod2vec

In [ ]:
# !pip install pandas --upgrade

In [11]:
from replib.descriptors.metaprod2vec import MetaProd2Vec

items = [
    {"title": "Apple iPhone 11"},
    {"title": "Samsung galaxy s9"}
]

descriptor = MetaProd2Vec(pre_fitted_version='MLA_autotrain_01_01_2022')
descriptor.transform(items).shape  # no need to train!

2022-03-30 15:54:33,080 - [WARNING] replib.core.descriptors.base:210 - Descriptor meta_prod2_vec was trained with package replib.descriptors.metaprod2vec==1.1.0 and you have installed replib.descriptors.metaprod2vec==1.2.3, which may be incompatible.
2022-03-30 15:54:33 Descriptor meta_prod2_vec was trained with package replib.descriptors.metaprod2vec==1.1.0 and you have installed replib.descriptors.metaprod2vec==1.2.3, which may be incompatible.


(2, 100)

In [16]:
from replib.descriptors.metaprod2vec import MetaProd2Vec

items = [
    {
        "title": "Apple iPhone 11 (64 GB) - (PRODUCT)RED",
        "product_id": "MLA15149566",
    },
    {
        "title": "Samsung Galaxy S10 128 GB Azul prisma 8 GB RAM",
    },
]

descriptor = MetaProd2Vec(pre_fitted_version='MLA_autotrain_01_01_2022')
descriptor.transform(items).shape

2022-03-30 16:01:10,612 - [WARNING] replib.core.descriptors.base:210 - Descriptor meta_prod2_vec was trained with package replib.descriptors.metaprod2vec==1.1.0 and you have installed replib.descriptors.metaprod2vec==1.2.3, which may be incompatible.
2022-03-30 16:01:10 Descriptor meta_prod2_vec was trained with package replib.descriptors.metaprod2vec==1.1.0 and you have installed replib.descriptors.metaprod2vec==1.2.3, which may be incompatible.


(2, 100)

In [12]:
variables = descriptor.transform(items)

In [13]:
variables

array([[ 8.62778187e-01,  6.20687366e-01,  1.69396400e-03,
         5.08175194e-02,  3.60768497e-01,  4.81142163e-01,
        -1.74911022e+00,  7.96282589e-01, -4.75240678e-01,
        -4.14445221e-01,  2.57109553e-02, -1.82441032e+00,
        -7.42173374e-01, -3.78606021e-02,  7.78555393e-01,
         1.78419924e+00,  2.28432536e-01,  9.12054360e-01,
         5.50023973e-01,  8.78153801e-01,  6.85774088e-01,
         3.43297064e-01,  6.74590707e-01,  5.27367413e-01,
         4.37868774e-01, -3.52018327e-03, -8.25429320e-01,
        -5.92662334e-01, -6.83253646e-01,  3.84581685e-01,
        -1.20679045e+00,  1.26034647e-01, -5.10308325e-01,
         1.56336105e+00,  8.07640433e-01, -2.81661749e-01,
         1.63797760e+00,  1.13016033e+00,  1.27159882e+00,
        -4.55961525e-01,  3.92147332e-01, -1.26867843e+00,
        -1.13733459e+00, -7.49039769e-01, -2.56188005e-01,
        -6.34409904e-01,  8.54350209e-01,  1.48442775e-01,
         1.06472909e+00, -1.43136609e+00, -3.74604166e-0

In [ ]:
site = conf['SITE']
backbone_table = conf['BACKBONE_TABLE_NAME']
time = ('2022-01-05', '2022-02-23')

In [ ]:
master_vars_path = conf['MASTERS_VARS_PATHS']
mater_path_gcp = conf['MASTER_BUCKET_PATH']

prj, bu, model = conf['PROYECT'],conf['BUSINESS_UNIT'],conf['MODEL']
bucket_path = f'{prj}/{bu}/{model}/{site}/'

#temporal_dir = random_name(10)

#### TARGET AND MODEL POPULATION

In [ ]:
#Model population.
df = bq.execute_response(f'SELECt * FROM {backbone_table}_{site}')

In [ ]:
df.head(3)

In [ ]:
df.info()

In [ ]:
df['SENT_DATE'] = pd.to_datetime(df.SENT_DATE)
df['CONVERSION'] = df['CONVERSION'].astype(np.int64)

In [ ]:
df.info()

In [ ]:
df.head(3)

In [ ]:
df.groupby('SENT_DATE').CUS_CUST_ID.count().head()

In [ ]:
df.groupby('SENT_DATE').CUS_CUST_ID.nunique().plot.bar(figsize=(16,4))
plt.title('Total user in test and control time series')
plt.grid()
#Remover el 2022-02-16
a = df.groupby('SENT_DATE').CUS_CUST_ID.nunique().mean()
plt.hlines(y = a, xmin =0,xmax =60,colors = 'k')

In [ ]:
print(f'Average push per day {np.round(a,0)}')

In [ ]:
df.groupby('SENT_DATE').CONVERSION.mean().head(10)*100

In [ ]:
(df.groupby('SENT_DATE').CONVERSION.mean()*100).plot.bar(figsize=(16,4))
plt.grid()
plt.title('Conversion time series')
plt.ylim((0,6))

In [ ]:
print(f'Mean conversion rate: {df.CONVERSION.mean()*100}%')

In [ ]:
df.groupby(['SENT_DATE','EVENT_TYPE']).CONVERSION.mean().head()

In [ ]:
df.groupby(['SENT_DATE','EVENT_TYPE']).CUS_CUST_ID.count().head()

In [ ]:
d = df.groupby(['SENT_DATE','EVENT_TYPE']).CUS_CUST_ID.count().reset_index()

In [ ]:
d.loc[d.EVENT_TYPE=='sent'].plot.bar(x = 'SENT_DATE', y = 'CUS_CUST_ID',figsize=(16,4))
plt.grid()

In [ ]:
d.loc[d.EVENT_TYPE=='control'].plot.bar(x = 'SENT_DATE', y = 'CUS_CUST_ID',figsize=(16,4))
plt.grid()

In [ ]:
prc = d.loc[d.EVENT_TYPE=='control'].CUS_CUST_ID.sum()*100/d.loc[d.EVENT_TYPE=='sent'].CUS_CUST_ID.sum()
print(f'Average percentage of control (pushed users) per sent: {np.round(prc,2)}%')

#### PUSH VARS.
- Amount of push notifiations in the last 7 days
- Recency from the last push

In [ ]:
#PUSH VARS
temporal_dir = random_name(n=10)
temporal_subdir = f"{temporal_dir}/{conf['PREFIX_MASTERS']['PUSH_VARS']}"
os.makedirs(f"{temporal_subdir}",exist_ok=True)
logger.info(f"Temporal subdir created: {temporal_subdir}.")

In [ ]:
download_(temporal_dir = temporal_subdir,
          bucket_path = bucket_path,
          bucket_var = master_vars_path['PUSH_VARS'],
          is_prod = False)

In [ ]:
push_df = pd.read_parquet(f'{temporal_subdir}/000000000000.parquet')

In [ ]:
push_df.head(3)

In [ ]:
def proc_pushvars(df:pd.DataFrame)->pd.DataFrame:
    """Pre-processing procedure for push notification variables.
    df: DataFrame with the variables.
    """
    logger.info(f"--------- START PROCESSING PROCEDURE ON PUSHVAR ---------")
    logger.info(f"INPUT DATAFRAME {df.shape} unique users {df.CUS_CUST_ID.nunique()}")
    
    df['SENT_DATE'] = pd.to_datetime(df.SENT_DATE)
    
    recency_na = np.round(df['RECENCY'].isna().mean(),3)*100
    logger.info(f"FILL RECENCY NA VALUES ({recency_na} %) WITH MAX(RECENCY)+1")
    df['RECENCY'] = df['RECENCY'].fillna(np.max(df.RECENCY)+1).astype(np.int64)
    
    r_push_na = np.round(df['R_PUSH_LAST_15D_VS_REST_OF_MONTH'].isna().mean(),3)*100
    logger.info(f"FILL R_PUSH_LAST_15D_VS_REST_OF_MONTH NA ({r_push_na}) VALUES WITH -1")
    df['R_PUSH_LAST_15D_VS_REST_OF_MONTH'] = df['R_PUSH_LAST_15D_VS_REST_OF_MONTH']\
    .fillna(-1).astype(np.int64)
    
    logger.info(f"ADDING VAR R_PUSH_RANK & TOTAL_PUSH_LAST_7D_RANK")
    df['R_PUSH_RANK'] = df['R_PUSH_LAST_15D_VS_REST_OF_MONTH'].rank(na_option='bottom',pct=True)
    df['TOTAL_PUSH_LAST_7D_RANK'] = df['TOTAL_PUSH_LAST_7D'].rank(na_option='bottom',pct=True)
    
    df = prefix_source_info(df = df,
                            prefix = conf['PREFIX_MASTERS']['PUSH_VARS'],
                            exceptions= ['Y','W','SENT_DATE','CUS_CUST_ID'])
    
    logger.info(f"OUTPUT DATAFRAME {df.shape} unique users {df.CUS_CUST_ID.nunique()}")
    logger.info(f"--------- END PROCESSING PROCEDURE ON PUSHVAR ---------")
    return df

In [ ]:
push_df = proc_pushvars(push_df)

In [ ]:
push_df.info()

In [ ]:
push_df.PUSHVAR_R_PUSH_RANK

In [ ]:
push_df['SENT_DATE'] = pd.to_datetime(push_df.SENT_DATE)
push_df['RECENCY'] = push_df['RECENCY'].fillna(np.max(push_df.RECENCY)+1).astype(np.int64)

In [ ]:
push_df.info()

In [ ]:
push_df['PUSHVAR_TOTAL_PUSH_LAST_7D'].value_counts(normalize=True).sort_index()

In [ ]:
push_df['PUSHVAR_TOTAL_PUSH_LAST_7D'].value_counts(normalize=True).sort_index().plot.bar(figsize=(16,4))
plt.grid()

In [ ]:
push_df['PUSHVAR_RECENCY'].value_counts(normalize=True).sort_index()

In [ ]:
push_df['PUSHVAR_RECENCY'].value_counts(normalize=True).sort_index().plot.bar(figsize=(16,4))
plt.grid()
#El valor de 8 es la imputacion. 

In [ ]:
push_df.shape

In [ ]:
push_df = prefix_source_info(df = push_df,
                             prefix = conf['PREFIX_MASTERS']['PUSH_VARS'],
                             exceptions= ['Y','W','SENT_DATE','CUS_CUST_ID'])

In [ ]:
# def proc_pushvars(df:pd.DataFrame)->pd.DataFrame:
#     """Pre-processing procedure for push notification variables.
#     df: DataFrame with the variables.
#     """
#     logger.info(f"START PROCESSING PROCEDURE ON PUSHVAR")
#     logger.info(f"INPUT DATAFRAME {df.shape} unique users {df.CUS_CUST_ID.nunique()}")
    
#     df['SENT_DATE'] = pd.to_datetime(df.SENT_DATE)
#     logger.info(f"FILL RECENCY NA VALUES WITH MAX(RECENCY)+1")
#     df['RECENCY'] = df['RECENCY'].fillna(np.max(df.RECENCY)+1).astype(np.int64)
    
#     df = prefix_source_info(df = df,
#                              prefix = conf['PREFIX_MASTERS']['PUSH_VARS'],
#                              exceptions= ['Y','W','SENT_DATE','CUS_CUST_ID'])
#     return df

In [ ]:
push_df.head()

#### BEHAVIOUR_VARS.

In [ ]:
#BEHAVIOUR_VARS
temporal_subdir = f"{temporal_dir}/{conf['PREFIX_MASTERS']['BEHAVIOUR_VARS']}"
os.makedirs(f"{temporal_subdir}",exist_ok=True)
logger.info(f"Temporal subdir created: {temporal_subdir}.")

In [ ]:
download_(temporal_dir = temporal_subdir,
          bucket_path = bucket_path,
          bucket_var = master_vars_path['BEHAVIOUR_VARS'],
          is_prod = False)

In [ ]:
beha_df = pd.read_parquet(f'{temporal_dir}/BEHAVIOURVARS/000000000000.parquet')

In [ ]:
beha_df.describe()

In [ ]:
series = beha_df.OS_NAME

In [ ]:
x = np.round(series.isnull().mean(),2)

In [ ]:
logger.info(f"% OF NaN in {series.name}: {x} %")

In [ ]:
class_p = series.value_counts(normalize=True).to_dict()

In [ ]:
proportions=class_p

In [ ]:
rv = np.random.choice(a = [*proportions.keys()],
                      size = series.isna().sum(),
                      p = [*proportions.values()]
                     ) 

In [ ]:
rv

In [ ]:
ix = np.arange(0,len(series))
series_ = series.copy()
for x,y,z in zip(ix,series.isna(),rv):
    if y:
        series_.iloc[x] = z

In [ ]:
series = beha_df.OS_NAME
x = np.round(series.isnull().mean(),2)
logger.info(f"% OF NaN in {series.name}: {x}")
rv = np.random.choice(a = [*proportions.keys()],
                      size = series.isna().sum(),
                      p = [*proportions.values()]
                     ) 
series.loc[series.isna()] = rv

In [ ]:
['LOG_DATE_COUNT_7D', 'AVG_PRICE_ITEM_7D','CAT_CATEG_ID_COUNT_7D', 'AVG_PRICE_ITEM_F7D',
       'MIN_PRICE_ITEM', 'MAX_PRICE_ITEM', 'BUY_INTENTION_ITEM_COUNT',
       'CAT_CATEG_ID_COUNT_F7D', 'VIP_VIEW_APP_ANDROID_ITEM_COUNT',
       'VIP_VIEW_APP_IOS_ITEM_COUNT', 'VIP_VIEW_DESKTOP_ITEM_COUNT',
       'VIP_VIEW_WEB_MOBILE_ITEM_COUNT', 'VIP_VIEW', 'QUESTION_ITEM_COUNT',
       'CHECKOUT_CONGRATS_ITEM_COUNT', 'ADD_TO_CART_ITEM_COUNT',
       'BOOKMARK_ITEM_COUNT', 'TOTAL_PAYMENTS_AMT_SUM',
       'WALLET_PAYMENTS_AMT_SUM', 'AGGREGATOR_PAYMENTS_AMT_SUM',
       'TOTAL_PAYMENTS_COUNT']

In [ ]:
beha_df.max(axis =0)

In [ ]:
beha_df.columns

In [ ]:
beha_df.info()

In [ ]:
#CASTEOS.
beha_df['SENT_DATE'] = pd.to_datetime(beha_df.SENT_DATE)
beha_df['MIN_PRICE_ITEM'] = beha_df['MIN_PRICE_ITEM'].astype(np.float64)
beha_df['MAX_PRICE_ITEM'] = beha_df['MAX_PRICE_ITEM'].astype(np.float64)
beha_df['TOTAL_PAYMENTS_AMT_SUM'] = beha_df['TOTAL_PAYMENTS_AMT_SUM'].astype(np.float64)
beha_df['WALLET_PAYMENTS_AMT_SUM'] = beha_df['WALLET_PAYMENTS_AMT_SUM'].astype(np.float64)
beha_df['AGGREGATOR_PAYMENTS_AMT_SUM'] = beha_df['AGGREGATOR_PAYMENTS_AMT_SUM'].astype(np.float64)
beha_df['AVG_PRICE_ITEM_7D'] = beha_df['AVG_PRICE_ITEM_7D'].astype(np.float64)
beha_df['AVG_PRICE_ITEM_F7D'] = beha_df['AVG_PRICE_ITEM_F7D'].astype(np.float64)

In [ ]:
beha_df.info()

In [ ]:
beha_df.head(3)

In [ ]:
beha_df['MIN_PRICE_ITEM']

In [ ]:
beha_df['MIN_PRICE_ITEM'] = beha_df['MIN_PRICE_ITEM'].astype(np.float64)
beha_df.boxplot(column = 'MIN_PRICE_ITEM')

In [ ]:
def outlier_clipping(series:pd.Series,q:float = .95):
    """Replace the values over the percentile 95 with the value
    of this percentile."""
    
    top = np.nanquantile(series,q)
    logger.info(f"{series.name} - MAX : {np.max(series)} WILL BE REPALCED BY {top}")
    series.loc[series > top] = top
    
    return series        

In [ ]:
beha_df['MIN_PRICE_ITEM'] = beha_df['MIN_PRICE_ITEM'].astype(np.float64)
beha_df['MAX_PRICE_ITEM'] = beha_df['MAX_PRICE_ITEM'].astype(np.float64)
for col in ['MIN_PRICE_ITEM','MAX_PRICE_ITEM']:
    beha_df[col] =  outlier_clipping(beha_df[col].copy(),q = .95)

In [ ]:
beha_df.boxplot(column = 'MIN_PRICE_ITEM')

In [ ]:
beha_df.boxplot(column = 'MAX_PRICE_ITEM')

In [ ]:
beha_df.info()

In [ ]:
cols = ['MIN_PRICE_ITEM','MAX_PRICE_ITEM',
        'MAX_FROM_INSTALL_ML','MIN_FROM_INSTALL_ML',
        'MAX_FROM_INSTALL_MP','MIN_FROM_INSTALL_MP']
for col in cols:
    logger.info(f"FILL NA {col} ({np.round(beha_df[col].isna().mean(),2)*100}%) VALUES WITH -1")
    beha_df[col] = beha_df[col].fillna(-1)

In [ ]:
beha_df['SENT_DATE'] = pd.to_datetime(beha_df.SENT_DATE)
beha_df['MIN_PRICE_ITEM'] = beha_df['MIN_PRICE_ITEM'].astype(np.float64)
beha_df['MAX_PRICE_ITEM'] = beha_df['MAX_PRICE_ITEM'].astype(np.float64)
beha_df['TOTAL_PAYMENTS_AMT_SUM'] = beha_df['TOTAL_PAYMENTS_AMT_SUM'].astype(np.float64)
beha_df['WALLET_PAYMENTS_AMT_SUM'] = beha_df['WALLET_PAYMENTS_AMT_SUM'].astype(np.float64)
beha_df['AGGREGATOR_PAYMENTS_AMT_SUM'] = beha_df['AGGREGATOR_PAYMENTS_AMT_SUM'].astype(np.float64)
beha_df['AVG_PRICE_ITEM_7D'] = beha_df['AVG_PRICE_ITEM_7D'].astype(np.float64)
beha_df['AVG_PRICE_ITEM_F7D'] = beha_df['AVG_PRICE_ITEM_F7D'].astype(np.float64)

beha_df['RECENCY'] = beha_df['RECENCY'].fillna(np.max(beha_df.RECENCY) + 1).astype(np.int64)

#carefull
beha_df['MIN_PRICE_ITEM'] = beha_df['MIN_PRICE_ITEM'].fillna(-1)
beha_df['MAX_PRICE_ITEM'] = beha_df['MAX_PRICE_ITEM'].fillna(-1)
beha_df['MAX_FROM_INSTALL_ML'] = beha_df['MAX_FROM_INSTALL_ML'].fillna(-1)
beha_df['MIN_FROM_INSTALL_ML'] = beha_df['MIN_FROM_INSTALL_ML'].fillna(-1)
beha_df['MAX_FROM_INSTALL_MP'] = beha_df['MAX_FROM_INSTALL_MP'].fillna(-1)
beha_df['MIN_FROM_INSTALL_MP'] = beha_df['MIN_FROM_INSTALL_MP'].fillna(-1)

In [ ]:
beha_df.info()

In [ ]:
beha_df['OS_NAME'].value_counts(normalize=True)

In [ ]:
class_p = beha_df['OS_NAME'].value_counts(normalize=True).to_dict()
categorical_imputer(series = beha_df['OS_NAME'],proportions = class_p)

In [ ]:
beha_df['OS_NAME'].value_counts(normalize=True)

In [ ]:
beha_df['OS_IS_ANDROID'] = remap_column(serie = beha_df['OS_NAME'],dict_ = {'android':1, 'ios':0})

In [ ]:
beha_df['OS_IS_ANDROID'].value_counts(normalize=True)

In [ ]:
beha_df.drop(columns='OS_NAME', inplace=True)

In [ ]:
beha_df = prefix_source_info(df = beha_df,
                             prefix = conf['PREFIX_MASTERS']['BEHAVIOUR_VARS'],
                             exceptions= ['Y','W','SENT_DATE','CUS_CUST_ID'])

In [ ]:
beha_df.shape

In [ ]:
beha_df.head()

In [ ]:
 beha_df.BEHAVIOURVARS_OS_NAME

In [ ]:
x = np.round(series.isnull().mean(),2)
logger.info(f"% OF NaN in {series.name}: {x}")
rv = np.random.choice(a = [*proportions.keys()],
                      size = series.isna().sum(),
                      p = [*proportions.values()]
                     ) 
series.loc[series.isna()] = rv

In [ ]:
def proc_behaviour(df: pd.DataFrame)->pd.DataFrame:
    """Pre-processing procedure for events on :
    marketplace and MP app.
    df: DataFrame with the variables.
    """
    logger.info(f"--------- START PROCESSING PROCEDURE ON BEHAVIOUR ---------")
    logger.info(f"INPUT DATAFRAME {df.shape} unique users {df.CUS_CUST_ID.nunique()}")
    
    logger.info(f"CAST VARIABLES TYPE")
    df['SENT_DATE'] = pd.to_datetime(df.SENT_DATE)
    df['MIN_PRICE_ITEM'] = df['MIN_PRICE_ITEM'].astype(np.float64)
    df['MAX_PRICE_ITEM'] = df['MAX_PRICE_ITEM'].astype(np.float64)
    df['TOTAL_PAYMENTS_AMT_SUM'] = df['TOTAL_PAYMENTS_AMT_SUM'].astype(np.float64)
    df['WALLET_PAYMENTS_AMT_SUM'] = df['WALLET_PAYMENTS_AMT_SUM'].astype(np.float64)
    df['AGGREGATOR_PAYMENTS_AMT_SUM'] = df['AGGREGATOR_PAYMENTS_AMT_SUM'].astype(np.float64)
    df['AVG_PRICE_ITEM_7D'] = df['AVG_PRICE_ITEM_7D'].astype(np.float64)
    df['AVG_PRICE_ITEM_F7D'] = df['AVG_PRICE_ITEM_F7D'].astype(np.float64)
    
    logger.info(f"Clipping prices and money by percentil 95")
    cols = ['AVG_PRICE_ITEM_7D','CAT_CATEG_ID_COUNT_7D', 
            'AVG_PRICE_ITEM_F7D','MIN_PRICE_ITEM', 'MAX_PRICE_ITEM', 
            'BUY_INTENTION_ITEM_COUNT','CAT_CATEG_ID_COUNT_F7D', 
            'VIP_VIEW_APP_ANDROID_ITEM_COUNT','VIP_VIEW_APP_IOS_ITEM_COUNT',
            'VIP_VIEW_DESKTOP_ITEM_COUNT','VIP_VIEW_WEB_MOBILE_ITEM_COUNT', 
            'VIP_VIEW', 'QUESTION_ITEM_COUNT','CHECKOUT_CONGRATS_ITEM_COUNT',
            'ADD_TO_CART_ITEM_COUNT','BOOKMARK_ITEM_COUNT', 
            'TOTAL_PAYMENTS_AMT_SUM','WALLET_PAYMENTS_AMT_SUM', 
            'AGGREGATOR_PAYMENTS_AMT_SUM','TOTAL_PAYMENTS_COUNT']
    for col in cols:
        df[col] =  outlier_clipping(df[col].copy(),q = .95)
    
    recency_na = np.round(df['RECENCY'].isna().mean(),3)*100
    logger.info(f"FILL RECENCY NA VALUES ({recency_na} %) WITH MAX(RECENCY)+1")
    df['RECENCY'] = df['RECENCY'].fillna(np.max(df.RECENCY) + 1).astype(np.int64)

    cols = ['MIN_PRICE_ITEM','MAX_PRICE_ITEM',
            'MAX_FROM_INSTALL_ML','MIN_FROM_INSTALL_ML',
            'MAX_FROM_INSTALL_MP','MIN_FROM_INSTALL_MP']
    for col in cols:
        logger.info(f"FILL NA {col} ({np.round(df[col].isna().mean(),2)*100}%) VALUES WITH -1")
        df[col] = df[col].fillna(-1)
    
    od_na = np.round(df['OS_NAME'].isna().mean(),3)*100
    logger.info(f"FILL NA VALUES IN OS_NAME ({od_na}%) BY ORIGINAL DISTIRBUTION")
    class_p = df['OS_NAME'].value_counts(normalize=True).to_dict()
    categorical_imputer(series = df['OS_NAME'],proportions = class_p)
    df['OS_IS_ANDROID'] = remap_column(serie = df['OS_NAME'],dict_ = {'android':1, 'ios':0})
    df.drop(columns='OS_NAME', inplace=True)
    
    df = prefix_source_info(df = df,
                            prefix = conf['PREFIX_MASTERS']['BEHAVIOUR_VARS'],
                            exceptions= ['Y','W','SENT_DATE','CUS_CUST_ID'])
    
    logger.info(f"OUTPUT DATAFRAME {df.shape} unique users {df.CUS_CUST_ID.nunique()}")
    logger.info(f"--------- END PROCESSING PROCEDURE ON BEHAVIOUR ---------")
    return df

In [ ]:
beha_df = proc_behaviour(beha_df)

#### MERGE.

In [ ]:
master = df.merge(push_df,on = ['CUS_CUST_ID','SENT_DATE'], how = 'left', validate='1:1')
master = master.merge(beha_df,on = ['CUS_CUST_ID','SENT_DATE'], how = 'left', validate='1:1')

In [ ]:
master.head(3)

In [ ]:
master.info()

In [ ]:
master['W'] = remap_column(serie = master['EVENT_TYPE'],dict_ = {'sent':1, 'control':0})
master.drop(columns='EVENT_TYPE', inplace=True)
master.rename(columns={"CONVERSION": "Y"}, inplace=True)

In [ ]:
master.info()

In [ ]:
master.W.value_counts(normalize=True)

In [ ]:
master.Y.value_counts(normalize=True)